# Import libraries

[Paper reference](https://ieeexplore.ieee.org.remotexs.ntu.edu.sg/stamp/stamp.jsp?tp=&arnumber=8861550&tag=1)

In [1]:
import pandas as pd
import pickle
from copy import deepcopy
import time
from tqdm.notebook import tqdm_notebook as pb

from IPython.core.display import display, HTML
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 100)
display(HTML("<style>.container { width:99% !important; }</style>"))

<ipython-input-1-0c9789ee8b2b>:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# General Functions

In [2]:
def dump_file(file_path, file_name, data_to_dump):
    import pickle
    with open(f'{file_path}/{file_name}.pkl', 'wb') as file:
        pickle.dump(data_to_dump, file)
        
def load_file(file_path, file_name):
    import pickle
    with open(f'{file_path}/{file_name}.pkl', 'rb') as file:
        output_file = pickle.load(file)
    return output_file

In [3]:
folder_path = '01_Paper_1'

In [58]:
# ========== save files ===============
dump_file(file_path=folder_path, file_name='stock_tickers', data_to_dump=stock_ticker)
dump_file(file_path=folder_path, file_name='stock_tickers_formatted_before_map', data_to_dump=stock_ticker_formatted)
dump_file(file_path=folder_path, file_name='stock_tickers_formatted', data_to_dump=stock_ticker_formatted)
dump_file(file_path=folder_path, file_name='tickers_map', data_to_dump=tickers_map)
dump_file(file_path=folder_path, file_name='financial_data_by_tickers', data_to_dump=financial_data_by_tickers)
dump_file(file_path=folder_path, file_name='financial_data_by_tickers_timeframe_final', data_to_dump=financial_data_by_tickers_timeframe_final)

# 1. Getting the dataset required

## Automating data download

In [55]:
stock_ticker = load_file(file_path=folder_path, file_name='stock_tickers')

In [56]:
stock_ticker_formatted_before_map = []
for st in stock_ticker:
    stock_ticker_formatted_before_map.append(st.split('-')[0].split(' ')[0])

In [57]:
# fixing the tickers to download the datasets, map to the correct ticker in request url

stock_ticker_formatted = deepcopy(stock_ticker_formatted_before_map)

tickers_map = {1: 'AMR', 7: 'ABI_0', 52: 'ABK', 63: 'TSO', 68: 'WLP', 69: 'AOC', 79: 'AA', 83: 'DV', 106: 'FO', 
               108: 'BF.B', 218: 'DG_1', 240: 'DOW', 241: 'CSC', 243: 'ERTS', 261: 'EL', 293: 'FRE', 294: 'FNM', 
              296:'NWS.A', 301: 'CZN', 311: 'WPO', 356: 'SLE', 362: 'IACI', 376: 'Q_1', 397: 'LIZ', 408: 'EK', 
              414: 'LTR', 415: 'LTD', 416: 'SAI', 418: 'LEH_0', 452: 'KFT', 480: 'MOT', 488: 'MNST_0', 498: 'FPL',
              569: 'PMTC', 636: 'MHP', 648: 'WFR', 668: 'GCI', 712: 'VIA.B', 713: 'JDSU', 732: 'WFMI', 738: 'WM', 
               739: 'WM_0', 762: 'ZMH'}

for n, st in enumerate(stock_ticker_formatted):
    if n in list(tickers_map.keys()):
        stock_ticker_formatted[n] = tickers_map[n]

In [94]:
# automate downloading of files using requests library
import requests
import pickle
import csv

tickers_not_retrieved = [] # may be file name error
financial_data_by_tickers = []
total_tickers = len(stock_ticker_formatted)

for n, st in enumerate(stock_ticker_formatted):
    url = f"https://web.archive.org/web/20180809015717/http://www.stockpup.com/data/{st}_quarterly_financial_data.csv"

    with requests.Session() as s:
        try:
            download = s.get(url)
            decoded_content = download.content.decode('utf-8')
            csv_lines = list(csv.reader(decoded_content.splitlines(), delimiter=','))
            df_financial_data_per_ticker = pd.DataFrame(csv_lines, columns=csv_lines[0]).drop(index=0)
            df_financial_data_per_ticker['ticker'] = st
            financial_data_by_tickers.append(df_financial_data_per_ticker)
            print(f'financial data of {n}/{total_tickers} retrieved, {st} appended')
        except Exception as e:
            print(f'ticker not retrieved {st} financial data of {n}/{total_tickers} NOT RETRIEVED', 'error:', e)
            tickers_not_retrieved.append(n)

financial data of 0/765 retrieved, A appended
financial data of 1/765 retrieved, AMR appended
financial data of 2/765 retrieved, AAN appended
financial data of 3/765 retrieved, AAP appended
financial data of 4/765 retrieved, AAPL appended
financial data of 5/765 retrieved, ABBV appended
financial data of 6/765 retrieved, ABC appended
financial data of 7/765 retrieved, ABI_0 appended
financial data of 8/765 retrieved, ABMD appended
financial data of 9/765 retrieved, ABT appended
financial data of 10/765 retrieved, ACAS appended
financial data of 11/765 retrieved, ACC appended
financial data of 12/765 retrieved, ACIW appended
financial data of 13/765 retrieved, ACM appended
financial data of 14/765 retrieved, ACN appended
financial data of 15/765 retrieved, ACS appended
financial data of 16/765 retrieved, ACV appended
financial data of 17/765 retrieved, ACXM appended
financial data of 18/765 retrieved, ADBE appended
financial data of 19/765 retrieved, ADCT appended
financial data of 20/7

# 2. Data Preprocessing

## Choosing the Stocks

In [7]:
financial_data_by_tickers = load_file(file_path=folder_path, file_name='financial_data_by_tickers')
stock_ticker_formatted_before_map = load_file(file_path=folder_path, file_name='stock_tickers_formatted_before_map')

In [30]:
# 102 leading stocks chosen. Assumption to be made that we find top 102 stocks that represents about 51% of market, then get the required timeline
df_top_200 = pd.read_csv(f'{folder_path}/top_200_stocks_by_components.csv')
top_200_list = list(df_top_200['Symbol'])


# check stocks in list
top_200_idx = []
for n, st in enumerate(stock_ticker_formatted_before_map):
    if st in top_200_list:
        top_200_idx.append(n)
        
# 95 stocks in top 102 in dataframes list
financial_data_by_tickers_final = []
for idx in top_200_idx:
    financial_data_by_tickers_final.append(financial_data_by_tickers[idx])
    
# out of 95 stocks take only the relevant quarters from Q1 1996 to Q4 2017
financial_data_by_tickers_timeframe = []
for df in financial_data_by_tickers_final:
    df_temp = df[(df['Quarter end'] >= '1996-03-29') & (df['Quarter end'] <= '2017-12-31')]
    financial_data_by_tickers_timeframe.append(df_temp)

In [37]:
# get only tickers with all 88 observations from Q1 1996 to Q4 2017 present, else drop them
for n, df in enumerate(financial_data_by_tickers_timeframe):
    if df.shape[0] != 88:
        del financial_data_by_tickers_timeframe[n]

In [38]:
# check shapes of dfs
for df in financial_data_by_tickers_timeframe:
    print(df.shape)

(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)
(88, 42)


In [51]:
# from dataframes that have all 88 observations, we select the top 70 then do feature dropping

# merge dataframe, get rank, sort them and select top 70

df_financial_data_top_97 = pd.concat(financial_data_by_tickers_timeframe). \
merge(df_top_200, left_on='ticker', right_on='Symbol'). \
sort_values(by='#', ascending=True). \
drop(columns = list(df_top_200.columns))

financial_data_by_tickers_timeframe_sorted = []
ticker_list = list(df_financial_data_top_97['ticker'].unique())
for t in ticker_list:
    df_temp = df_financial_data_top_97[df_financial_data_top_97['ticker'] == t]. \
    sort_values(by='Quarter end', ascending=True). \
    reset_index(drop=True)
    
    financial_data_by_tickers_timeframe_sorted.append(df_temp)
    
financial_data_by_tickers_timeframe_final = deepcopy(financial_data_by_tickers_timeframe_sorted[0:70])

## Feature Dropping

In [65]:
financial_data_by_tickers_timeframe_final = load_file(file_path=folder_path, file_name='financial_data_by_tickers_timeframe_final')

In [67]:
# check missing values
# An assumption is to select 21 features that have roughly all values present or are good features by logical reasoning and per research

# find out columns with 88 data points
cols_wanted = ['Assets', 'Current Assets', 'Liabilities', 'Current Liabilities', 'P/E ratio']
financial_data_by_tickers_timeframe_final[0]

,Quarter end,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,Shareholders equity,Non-controlling interest,Preferred equity,Goodwill & intangibles,Long-term debt,Revenue,Earnings,Earnings available for common stockholders,EPS basic,EPS diluted,Dividend per share,Cash from operating activities,Cash from investing activities,Cash from financing activities,Cash change during period,Cash at end of period,Capital expenditures,Price,Price high,Price low,ROE,ROA,Book value of equity per share,P/B ratio,P/E ratio,Cumulative dividends per share,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio,ticker
0,1996-03-29,123716810,3464070680,28,5234000000,4277000000,3178000000,2273000000,2056000000,0,0,0,303000000,2185000000,-740000000,-740000000,-5.99,None,0,-364000000,169000000,-129000000,-324000000,500000000,9000000,1.04,1.27,0.82,-0.2443,-0.1071,0.59,1.28,21.57,0.03,None,0.1474,0.3928,None,1.81,-0.11,1.8817,AAPL
1,1996-06-28,124478256,3485391168,28,5345000000,4454000000,3325000000,1926000000,2020000000,0,0,0,949000000,2179000000,-32000000,-32000000,-0.26,None,0,289000000,86000000,484000000,859000000,1359000000,15000000,0.86,1.03,0.7,-0.3197,-0.1337,0.58,1.46,None,0.03,None,0.4698,0.3779,None,1.8,0.08,2.3126,AAPL
2,1996-09-27,124552511,3487470308,28,5364000000,4515000000,3306000000,2003000000,2058000000,0,0,0,949000000,2321000000,25000000,25000000,0.21,None,0,407000000,-227000000,13000000,193000000,1552000000,12000000,0.73,0.89,0.57,-0.3656,-0.1451,0.59,1.26,None,0.03,None,0.4611,0.3837,None,1.75,0.11,2.2541,AAPL
3,1996-12-27,124669324,3490741072,28,5272000000,4419000000,3330000000,2044000000,1942000000,0,0,0,950000000,2129000000,-120000000,-120000000,-0.96,None,0,94000000,-470000000,-2000000,-378000000,1174000000,20000000,0.88,0.99,0.76,-0.4294,-0.1635,0.56,1.49,None,0.03,None,0.4892,0.3684,None,1.66,0.02,2.1619,AAPL
4,1997-03-28,126354086,3537914408,28,4487000000,3642000000,3245000000,2011000000,1242000000,0,0,0,952000000,1601000000,-708000000,-708000000,-5.64,None,0,100000000,41000000,-42000000,99000000,1273000000,16000000,0.69,0.83,0.54,-0.4599,-0.1632,0.35,1.23,None,0.03,None,0.7665,0.2768,None,1.61,0.02,1.811,AAPL
5,1997-06-27,127329661,3565230508,28,4341000000,3493000000,3145000000,1910000000,1196000000,0,0,0,951000000,1737000000,-56000000,-56000000,-0.44,None,0,-204000000,-48000000,-3000000,-255000000,1018000000,6000000,0.61,0.71,0.52,-0.5337,-0.1765,0.34,1.74,None,0.03,None,0.7952,0.2755,None,1.6,-0.06,1.8288,AAPL
6,1997-09-26,127993412,3583815536,28,4233000000,3424000000,3033000000,1818000000,1200000000,0,0,0,951000000,1614000000,-161000000,-161000000,-1.26,None,0,198000000,-56000000,70000000,212000000,1230000000,11000000,0.76,1.06,0.46,-0.7491,-0.228,0.33,2.24,None,0.03,None,0.7925,0.2835,None,1.54,0.05,1.8834,AAPL
7,1997-12-26,132768062,3717505736,28,4126000000,3373000000,2882000000,1669000000,1244000000,0,0,0,952000000,1578000000,47000000,47000000,0.37,0.33,0,143000000,-181000000,1000000,-37000000,1193000000,7000000,0.67,0.88,0.46,-0.7194,-0.2043,0.33,2.03,None,0.03,None,0.7653,0.3015,None,1.52,0.04,2.021,AAPL
8,1998-03-27,133040579,3725136212,28,3963000000,3213000000,2575000000,1384000000,1388000000,0,0,0,953000000,1405000000,55000000,55000000,0.42,0.38,0,153000000,-72000000,11000000,92000000,1285000000,5000000,0.73,1,0.46,-0.0915,-0.0276,0.37,2.21,None,0.03,None,0.6866,0.3502,None,1.52,0.04,2.3215,AAPL
9,1998-06-26,134639382,3769902696,28,4041000000,3375000000,2555000000,1389000000,1486000000,0,0,0,953000000,1402000000,101000000,101000000,0.76,0.65,0,156000000,-217000000,-21000000,-82000000,1203000000,17000000,1,1.13,0.88,0.0316,0.0103,0.39,2.7,None,0.03,0,0.6413,0.3677,0.007,1.47,0.04,2.4298,AAPL
